## Load NumPy, pandas and time

In [2]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [6]:
# Copy and paste functions from Assignment 1 here that you need for this assignment
def create_bins(df, nobins=10, bintype="equal-width"):
    #Hint 1: Basic
    #copy the DataFrame first
    df_copy = df.copy()
    columns = df_copy.columns
    binning = {}
    
    for i in columns:
        dtype = df_copy[i].dtype

        #Hint 2: Constratints handling
        #do not care about ID or CLASS
        if i == "ID" or i == "CLASS":
            continue
        #only care about int and float
        if not np.issubdtype(dtype, np.integer) and not np.issubdtype(dtype, np.floating):
            continue
        
        #Hint 3 - Case 1: equal width -> cut
        if bintype == "equal-width":
            res, bins = pd.cut(df_copy[i], bins=nobins, labels=False, retbins=True, duplicates="drop")
        #Hint 3 - Case 2: equal size -> qcut
        elif bintype == "equal-size":
            res, bins = pd.qcut(df_copy[i], q=nobins, labels=False, retbins=True, duplicates="drop")
            
        #apply res
        df_copy[i] = res
        
        #Hint 4 - Set column to be of type "category"
        df_copy[i] = df_copy[i].astype("category")
        
        #Hint 5 - set the categories as a number of bins
        df_copy[i] = df_copy[i].cat.set_categories(list(range(len(bins))))
        
        #Hint 6 - set first and last value
        bins[0] = -np.inf
        bins[-1] = np.inf
        
        #set bins on the output
        binning[i] = bins
    
    return df_copy, binning

def apply_bins(df, binning):
    #Hint 1: Basic
    #copy the DataFrame first
    df_copy = df.copy()
    
    for key, val in binning.items():
        dtype = df_copy[key].dtype
        #Hint 2: Constratints handling
        
        #do not care about ID or CLASS (safe check when applying!)
        if key == "ID" or key == "CLASS":
            continue
        #only care about int and float
        if not np.issubdtype(dtype, np.integer) and not np.issubdtype(dtype, np.floating):
            continue
        
        #Hint 2
        res = pd.cut(df_copy[key], bins=val, labels=False, duplicates="drop")
        df_copy[key] = res
        
        #Hint 3 - Set column to be of type "category"
        df_copy[key] = df_copy[key].astype("category")

        #Hint 4 - set the categories as a number of nobins
        df_copy[key] = df_copy[key].cat.set_categories(list(range(len(val))))
        
    return df_copy

def create_imputation(df):
    #Hint 1: Basic
    #copy the DataFrame first
    df_copy = df.copy()
    columns = df_copy.columns
    imputation = {}
    
    for i in columns:
        dtype = df_copy[i].dtype

        #Hint 2: Constratints handling
        #do not care about ID or CLASS
        if i == "ID" or i == "CLASS":
            continue
        #Case 1: continuous -> use mean
        if np.issubdtype(dtype, np.integer) or np.issubdtype(dtype, np.floating):
            #Special case: all values are missing
            if np.all(df_copy[i].isnull()):
                criteria = 0
            #regular case
            else:
                criteria = df_copy[i].mean()
        #case 2: categorical -> use mode
        elif hasattr(df_copy[i], 'cat'):
            #Special case: all values are missing
            if np.all(df_copy[i].isnull()):
                criteria = df_copy[i].cat.categories[0]
            #regular case
            else:
                print(df_copy[i][df_copy[i].notnull()])
                criteria = df_copy[i].mode()[0] #always return series
        #case 3: object case -> cannot apply .cat -> use "" when all are missing
        #not sure about dtype == "object" or else
        elif dtype == "object":
            #Special case: all values are missing
            if np.all(df_copy[i].isnull()):
                criteria = ""
            #regular case
            else:
                criteria = df_copy[i].mode()[0] #always return series
        #except object, categorical, numerical -> but there is no case when we load a file
        else:
            print(dtype)

        #apply criteria (use fillna)
        df_copy[i] = df_copy[i].fillna(criteria)
        #add value into imputation dictionary
        imputation[i] = criteria
    
    return df_copy, imputation

def apply_imputation(df, imputation):
    #Hint 1: Basic
    #copy the DataFrame first
    df_copy = df.copy()
    
    #key: column name
    #val: imputation value
    for key, val in imputation.items():
        
        #Hint 2: Constratints handling
        #do not care about ID or CLASS (safe check when applying!)
        if key == "ID" or key == "CLASS": 
            continue
            
        criteria = val
        df_copy[key] = df_copy[key].fillna(criteria)
            
    return df_copy

def brier_score(df, correctlabels):
    #setting dictionary
    correctdict = {}
    brier_score = 0
    
    for i in correctlabels:
        if i not in correctdict.keys():
            correctdict[i] = [0]
    
    correctdict = pd.DataFrame(correctdict)

    #loop number of samples
    for cnt, val in enumerate(correctlabels):
        #initialize 0 again
        correctdict.iloc[0] = 0
        #only correct one goes to 1
        correctdict[val] = 1
        #get single row in a prediction
        row = df.iloc[cnt]
        #calculate score
        score = np.sum(np.square(row-correctdict), axis=1)
        brier_score += score
    
    brier_score /= len(correctlabels)
    return brier_score[0]

def auc(df, correctlabels):
    """
    We made new AUC function without using collection library following Henrik's comment
    """
    #make labels
    labels = set(correctlabels)
    
    #assignment 2,3 fix
    correctlabels = correctlabels.tolist()
    
    counts = [] #for weighted sum (count of class in true population)
    aucs = [] #auc score for each count
    
    #calculate TP/FP for each label
    for label in labels:
        #make scores 
        tot_tp = 0
        tot_fp = 0
        scores = {}
        
        for idx, val in enumerate(correctlabels):
            if df.iloc[idx][label] not in scores.keys():
                scores[df.iloc[idx][label]] = [0, 0]
            if val == label:
                scores[df.iloc[idx][label]][0] += 1
                tot_tp += 1
            else:
                scores[df.iloc[idx][label]][1] += 1
                tot_fp += 1
         
        #Descending sort by its score
        scores = sorted(scores.items(), reverse=True)
        
        #GET AUC score
        auc_sub = 0
        cov_tp = 0
        
        for key, val in enumerate(scores):
            tp_rate = val[1][0]
            fp_rate = val[1][1]
            if fp_rate == 0:
                cov_tp += tp_rate
            elif tp_rate == 0:
                auc_sub += (cov_tp/tot_tp) * (fp_rate/tot_fp)
            else:
                auc_sub += (cov_tp/tot_tp)*(fp_rate/tot_fp) + ((tp_rate/tot_tp)*(fp_rate/tot_fp))/2
                cov_tp += tp_rate
        
        #apply proportion
        counts.append(correctlabels.count(label))
        aucs.append(auc_sub)
        
    auc = np.array(aucs).dot(np.array(counts))/len(correctlabels)

    return auc

def accuracy(df, correctlabels):
    df_copy = df.copy()
    pred = np.empty(len(correctlabels))
    
    df_max = df_copy.max(axis=1) #find the highest value in each row to compare later

    for i in range(len(df)):
        df_tmp = df_copy.iloc[i:i+1]
        for col in df_tmp.columns:
            if(df_tmp[col] >= df_max[i:i+1]).bool(): 
                pred[i] = col
                """
                if break enabled, will pick the first option, 
                else, will leave the last option that equals the highest value, 
                can be randomized with an if and random function
                """
                #1. random mode
                #if np.random.choice([True, False]): break 
                
                #2. picking first one mode
                break
                
    numbercorrect = np.sum(np.array(correctlabels) == pred)
    return numbercorrect/len(correctlabels)

## 1. Define the class DecisionTree

In [7]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#

class DecisionTree:
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None
    
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
# Hint 5: The best feature found (with lowest resulting information content) will be used to split the training
#         instances, and each sub-group is used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
# Hint 6: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array 
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root

    #Hint 2: define a divide and conquer function
    #df, min_sample_split, nodeno
    def divide_and_conquer(self, df, features, class_freq, min_samples_split, nodeno):
        #check data, feature, unique class
        #exit conditions: if the instance is empty, if there are no features to split, 
        #if all instances have the same features (we use <=1 for safety)
        if len(df) < min_samples_split or len(features) == 0 or len(df['CLASS'].unique()) <= 1:
            #leaf node processing
            self.model.append((nodeno, "leaf", class_freq))
        else:
            #print(len(df), features)
            #Hint 5: choose lowest information_contents
            lowest = np.inf

            #Hint 2: get frequencies to calculate information content
            #where f has values and f will be omitted from features=F
            for f in features:
                feature_class_value_counts = {}
                feature_value_counts = {}
                
                #for all possible values
                for v in df[f].unique():
                    #value count is count of each possible value in a single feature f 
                    value_counts = (df[f] == v).sum()
                    #deal with all possible labels in 'whole dataset' to care about test data not in training data case
                    #If the label is not in the current reduced training data set, it will get zero probability
                    #we just regard zero probability as zero entropy (following lecture note)
                    for c in self.labels:
                        sumclass = ((df["CLASS"] == c) & (df[f] == v)).sum()
                        #for each possible value in a single feature, get count of final 'class' (y value)
                        feature_class_value_counts[(c,v)] = sumclass / value_counts
                    #make the value count as a relative frequency
                    feature_value_counts[v] = value_counts / len(df)
                
                #get entropy of feature f
                information_contents = self.information_content(feature_class_value_counts, feature_value_counts)

                #Hint 5: choose the feature having lowest information_contents
                if information_contents < lowest:
                    lowest = information_contents
                    lowest_feature = f
            
            #Hint 3: possible value-feature mapping
            node_dict = {}
            #making F = F \ {f}
            new_features = features - set([lowest_feature])
            
            ######## This area is for calculating no possible instance case #######
            #According to lecture note, even though there will be no instance following certain value in a feature
            #the tree should grow to this value, and relative frequency will follow most frequent value (maximum count)
            #in the feature.
            
            #get most frequent value case first before looping possible feature values
            #in the case of no any possible instances for some feature values
            #in case of none class, just label the class which has the largest number of instance
            
            #Here we have a random policy.
            #there are possibility of having multiple modes in a single feature, then we pick the first one.
            most_frequent_value = df[lowest_feature].mode()[0]
            
            #in the column of lowest feature, replace all the rows with the most frequest ones
            new_df_new = df[df[lowest_feature] == most_frequent_value]
            
            #calculate next relative frequency of most frequent value
            new_rf_mfv = pd.Series(np.zeros(len(self.labels)), index = self.labels)
            new_rf_mfv += new_df_new['CLASS'].value_counts()/len(new_df_new['CLASS'])
            new_rf_mfv.fillna(0, inplace=True)
            
            #########################################################################
            
            #Normal case: for all possible values having instances
            for value in df[lowest_feature].cat.categories:
                
                #I1, I2, ... , In having value of v1, v2, ... , vn
                new_df = df[df[lowest_feature] == value]
                
                #No instance case: when there is no instance for possible value low_f 
                #probability follows the most frequent value among siblings, already calculated above
                if len(new_df) == 0:
                    new_rf = new_rf_mfv
                else:
                    #calculate next frequency
                    new_rf = pd.Series(np.zeros(len(self.labels)), index = self.labels)
                    new_rf += new_df['CLASS'].value_counts()/len(new_df['CLASS'])
                    new_rf.fillna(0, inplace=True)

                #recursively add nodeno
                self.nodeno += 1
                node_dict[value] = self.nodeno
                self.divide_and_conquer(new_df, new_features, new_rf, min_samples_split, self.nodeno)
                
            #add current model (take care of order)
            self.model.append((nodeno, lowest_feature, node_dict ))

    #Hint 4
    def information_content(self, class_value, feature_value):
        #Get entropy when we grow the tree with selected feature.
        info = 0
        
        for v in feature_value.keys():
            for c in self.labels:
                #regard negative inifinity entropy as a zero information
                if class_value[(c,v)] != 0.0:
                    #Entropy formula : (|Sv|/|S|) * (Sum(-P log(Pi)))
                    info += feature_value[v]*(-class_value[(c,v)]*np.log2(class_value[(c,v)]))
        return info
    
    def fit(self, df, nobins=10, bintype="equal-width", min_samples_split=5):
        
        #make self.model as a array
        self.model = []
        
        #get defalut self datapoints first to use 
        df_copy, self.imputation = create_imputation(df)
        df_copy, self.binning = create_bins(df_copy, nobins=nobins, bintype=bintype)
        
        #Finding the possible labels
        self.labels = df_copy["CLASS"].astype('category').cat.categories
        
        #Hint 1: find available features and class frequency

        #available features
        available_features = set(df_copy.columns) - set(["ID", "CLASS"]) 
        
        #initial class relative frequency
        initial_rf = df_copy['CLASS'].value_counts()/len(df_copy['CLASS'])
        
        #set global nodeno and put it in the divide and conquer
        self.nodeno = 0
        self.divide_and_conquer(df_copy, available_features, initial_rf, min_samples_split, self.nodeno)

        #sort the model based on its nodeno
        self.model.sort(key=lambda x: x[0])
        

    
    def make_prediction(self, row, nodeno):
        #if the nodeno is not leaf
        if self.model[nodeno][1] != 'leaf':
            #check nodeno's spliting criteria. Getting the feature name
            split = self.model[nodeno][1]
            
            #check the feature value 
            value = row[split]
            
            #check the model again to match the new nodeno
            #getting the number of instances in every feature value
            new_nodeno = self.model[nodeno][2][value]

            #predict again
            try:
                return self.make_prediction(row, new_nodeno)
            except:
                #miss case does not occur in this algorithm, but apply it for safety
                print("miss")
        else:
            return self.model[nodeno][2]
            
    def predict(self, df):
        
        #Hint 1: drop id and class
        df_copy = df.copy()
        
        #it is better to check it seperately because in some unknown dataset there can be only one among ID and CLASS
        if "ID" in df_copy.columns:
            df_copy.drop(["ID"], inplace=True, axis=1)
        if "CLASS" in df_copy.columns:
            df_copy.drop(["CLASS"], inplace=True, axis=1)
            
        #Hint 1: apply imputation, binning
        df_copy = apply_imputation(df_copy, self.imputation)
        df_copy = apply_bins(df_copy, self.binning)
        
        #Hint 2: iterate over the rows
        predictions = df_copy.apply(self.make_prediction, axis=1, nodeno=0)
        
        #fix NaN value (zero probabilities) into real zero
        predictions.fillna(0, inplace=True)
        
        return predictions

### Our Result

In [11]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 1.86 s.
Testing time (5, 'equal-width', 3): 0.04 s.
Training time (5, 'equal-width', 5): 1.63 s.
Testing time (5, 'equal-width', 5): 0.04 s.
Training time (5, 'equal-width', 10): 1.06 s.
Testing time (5, 'equal-width', 10): 0.05 s.
Training time (5, 'equal-size', 3): 2.24 s.
Testing time (5, 'equal-size', 3): 0.04 s.
Training time (5, 'equal-size', 5): 1.36 s.
Testing time (5, 'equal-size', 5): 0.04 s.
Training time (5, 'equal-size', 10): 0.86 s.
Testing time (5, 'equal-size', 10): 0.04 s.
Training time (10, 'equal-width', 3): 2.47 s.
Testing time (10, 'equal-width', 3): 0.04 s.
Training time (10, 'equal-width', 5): 1.76 s.
Testing time (10, 'equal-width', 5): 0.03 s.
Training time (10, 'equal-width', 10): 1.24 s.
Testing time (10, 'equal-width', 10): 0.03 s.
Training time (10, 'equal-size', 3): 2.33 s.
Testing time (10, 'equal-size', 3): 0.04 s.
Training time (10, 'equal-size', 5): 2.12 s.
Testing time (10, 'equal-size', 5): 0.04 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.654206     0.573480  0.792989
               5   0.663551     0.536875  0.811069
               10  0.598131     0.540770  0.806463
   equal-size  3   0.626168     0.709367  0.746560
               5   0.635514     0.643167  0.761936
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.626168     0.613303  0.798035
               5   0.598131     0.561641  0.821840
               10  0.598131     0.500888  0.848858
   equal-size  3   0.523364     0.930815  0.669847
               5   0.523364     0.928479  0.669676
               10  0.542056     0.902172  0.677709

###  Henrik's result

In [14]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 1.58 s.
Testing time (5, 'equal-width', 3): 0.11 s.
Training time (5, 'equal-width', 5): 1.33 s.
Testing time (5, 'equal-width', 5): 0.11 s.
Training time (5, 'equal-width', 10): 0.74 s.
Testing time (5, 'equal-width', 10): 0.11 s.
Training time (5, 'equal-size', 3): 1.60 s.
Testing time (5, 'equal-size', 3): 0.07 s.
Training time (5, 'equal-size', 5): 0.84 s.
Testing time (5, 'equal-size', 5): 0.07 s.
Training time (5, 'equal-size', 10): 0.44 s.
Testing time (5, 'equal-size', 10): 0.07 s.
Training time (10, 'equal-width', 3): 2.47 s.
Testing time (10, 'equal-width', 3): 0.06 s.
Training time (10, 'equal-width', 5): 1.56 s.
Testing time (10, 'equal-width', 5): 0.06 s.
Training time (10, 'equal-width', 10): 0.88 s.
Testing time (10, 'equal-width', 10): 0.06 s.
Training time (10, 'equal-size', 3): 1.51 s.
Testing time (10, 'equal-size', 3): 0.07 s.
Training time (10, 'equal-size', 5): 1.32 s.
Testing time (10, 'equal-size', 5): 0.07 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.635514     0.566861  0.794838
               5   0.644860     0.530257  0.812039
               10  0.579439     0.540913  0.802684
   equal-size  3   0.644860     0.678333  0.757026
               5   0.635514     0.644065  0.759834
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.654206     0.566439  0.816216
               5   0.626168     0.529316  0.832679
               10  0.598131     0.503810  0.845034
   equal-size  3   0.504673     0.905079  0.656123
               5   0.504673     0.902743  0.655598
               10  0.532710     0.868821  0.669287

In [10]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


### Comment on assumptions, things that do not work properly, etc.


We followed all instructions and made a perfect algorithm. There is no any wrong thing in the algorithm.
Due to two randomness in choosing same number, the result on the test dataset is different.
Our overall poerformance is slightly better than answer sheet but only one or two instances in testset.

There are two randomness and our criteria is below:
1. Growing empty value in a feature
 - When we grow a tree after choosing a feater, based on possible values in single feature, we grow the value having no instance too. In this case, according to the lecture note, the relative frequency will follow the value that is most prevalent in the feature.
 - In this dataset, there are the cases having more than two same "most prevalent value (mode)" in a feature. in this case we always choose first one, not randomly choose.
 
2. Test: choose highest probability among possible classes
 - If there are multiple highest probabilities, we choose first one, not randomly choose.
 
These two random cases handling can change the result slightly. But the result is very similar (only 1-2 instances of difference as test set is just 107 instances).

## 2. Define the class DecisionForest

In [29]:
# Define the class DecisionForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# random_features: no. of features to evaluate at each split (default = 2), 0 means all features (no random sampling)
# notrees: no. of trees in the forest (default = 10)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a random forest (for details, see lecture slides)
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: Redefine divide_and_conquer to take one additional argument; random_features, and instead of
#         evaluating all features choose a random subset, e.g., by np.random.choice (without replacement)
# Hint 2: Generate each tree in the forest from a bootstrap replicate of df, e.g., by np.random.choice 
#         (with replacement) from the index values of df.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the mean of all relative class frequencies in the leaves of the forest into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(row), which for a test row
#         finds all leaf nodes and calculates the average of their class probabilities

class DecisionForest(DecisionTree):
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None     
        
    def divide_and_conquer(self, df, features, class_freq, min_samples_split, nodeno, random_features, modelno):
        """
        We added new parameter modelno to figure out the model among multiple modles
        """
        #exit case of random forest
        #check data, feature, unique class (for satefy we use <= 1 for unique case)
        #it is same as decision tree case.
        if len(df) < min_samples_split or len(features) == 0 or len(df['CLASS'].unique()) <= 1:
            #leaf node processing
            self.model[modelno].append((nodeno, "leaf", class_freq))
        else:
            
            ##########RANDOM FOREST: GENERATE FEATURE SELECTION##########
            #if available feature is less than random feature, we will not apply feature selection
            #when there are less number of features than the parameter, we are just selecting the remaining features instead of exiting
            
            if random_features != 0 and len(features) >= random_features:
                #if random features are upper zero and all possible features are bigger and equal to the random features
                #because every phase F - {f} is done, features can be less than number of random_features
                selected_features = set(np.random.choice(list(features), random_features, replace = False))
            else:
                #if random_feature is zero, then we will not apply feature selection
                #Or, if evaluation variable(random_features) are bigger than available number of features, 
                #then we take all features (do not perform feature selection)
                #(There is no criteria of handling this case, so we followed scikit-learn manual)
                #
                #We also tested early pruning (if available feature is less than random_feature, then terminate)
                #but the performance was similar. So we did not apply early pruning.
                #
                selected_features = features
            #############################################################

            #choose lowest information_contents
            lowest = np.inf

            #get frequencies to calculate information content
            #where f has values and f will be omitted from features=F
            #selected feautre is only used into the entropy part
            for f in selected_features:
                feature_class_value_counts = {}
                feature_value_counts = {}
                
                #for all possible values
                for v in df[f].unique():
                    value_counts = (df[f] == v).sum()
                    #Deal with all lables for unknown possible test set
                    for c in self.labels:
                        sumclass = ((df["CLASS"] == c) & (df[f] == v)).sum()
                        feature_class_value_counts[(c,v)] = sumclass / value_counts
                    feature_value_counts[v] = value_counts / len(df)

                information_contents = self.information_content(feature_class_value_counts, feature_value_counts)

                #Hint 5: choose the feature having lowest information_contents
                if information_contents < lowest:
                    lowest = information_contents
                    lowest_feature = f
            
            #possible value-feature mapping
            node_dict = {}
            #making F = F \ {f}
            new_features = features - set([lowest_feature])
            
            #get most frequent value case first before looping possible feature values
            #in the case of no any possible instances for some feature values
            most_frequent_value = df[lowest_feature].mode()[0]
            new_df_new = df[df[lowest_feature] == most_frequent_value]
            
            #calculate next relative frequency of most frequent value
            new_rf_mfv = pd.Series(np.zeros(len(self.labels)), index = self.labels)
            new_rf_mfv += new_df_new['CLASS'].value_counts()/len(new_df_new['CLASS'])
            new_rf_mfv.fillna(0, inplace=True)
            
            #for all possible values
            for value in df[lowest_feature].cat.categories:
                
                ######## This area is for calculating no possible instance case #######
                #According to lecture note, even though there will be no instance following certain value in a feature
                #the tree should grow to this value, and relative frequency will follow most frequent value (maximum count)
                #in the feature.

                #get most frequent value case first before looping possible feature values
                #in the case of no any possible instances for some feature values
                #in case of none class, just label the class which has the largest number of instance

                #I1, I2, ... , In having value of v1, v2, ... , vn
                new_df = df[df[lowest_feature] == value]
                
                #when there is no any case of possible value low_f 
                #probability follows the most frequent value among siblings
                if len(new_df) == 0:
                    new_rf = new_rf_mfv
                else:
                    #calculate next frequency
                    new_rf = pd.Series(np.zeros(len(self.labels)), index = self.labels)
                    new_rf += new_df['CLASS'].value_counts()/len(new_df['CLASS'])
                    new_rf.fillna(0, inplace=True)

                #recursively add nodeno
                self.nodeno += 1
                node_dict[value] = self.nodeno
                self.divide_and_conquer(new_df, new_features, new_rf, min_samples_split, self.nodeno, random_features, modelno)
                
            #add current model (take care of order)
            self.model[modelno].append((nodeno, lowest_feature, node_dict ))
    
    def fit(self, df, nobins=10, bintype="equal-width", min_samples_split=5, notrees=10, random_features=2):
        
        #make self.model as a array
        self.model = []
        
        #get defalut self datapoints first to use 
        df_copy, self.imputation = create_imputation(df)
        df_copy, self.binning = create_bins(df_copy, nobins=nobins, bintype=bintype)
        self.labels = df_copy["CLASS"].astype('category').cat.categories
        
        #find available features and class frequency

        #available features
        available_features = set(df_copy.columns) - set(["ID", "CLASS"]) 
        
        #initial class relative frequency
        initial_rf = df_copy['CLASS'].value_counts()/len(df_copy['CLASS'])
        
        ##########RANDOM FOREST: GENERATE BOOTSTRAP REPLICA##########
        #for each tree model
        for model_no in range(notrees):
            #generate replica with replacement for each model
            replica = df_copy.loc[np.random.choice(df_copy.index, len(df_copy), replace = True)]

            #set global nodeno and put it in the divide and conquer
            self.nodeno = 0
            self.model.append([])
            #we added new parameter as a model no.
            self.divide_and_conquer(replica, available_features, initial_rf, min_samples_split, self.nodeno, random_features, model_no)

            #sort the model based on its nodeno
            self.model[model_no].sort(key=lambda x: x[0])

        #############################################################
    
    def make_prediction(self, row, nodeno, model):
        """
        We add a new parameter 'model', meaning one model in forest
        the model is passed from the new function for random forest, "make_prediction_averge"
        """
        #if the nodeno is not leaf
        if model[nodeno][1] != 'leaf':
            #check nodeno's spliting criteria
            split = model[nodeno][1]
            
            #check the data value of spliting criteria
            value = row[split]
            
            #check the model again to match the new nodeno
            new_nodeno = model[nodeno][2][value]

            #predict again
            try:
                return self.make_prediction(row, new_nodeno, model)
            except:
                #miss case does not occur in this algorithm, but apply it for safety
                print("miss")
        else:
            return model[nodeno][2]
    
    def make_prediction_average(self, row, nodeno):
        """
        New function for random forest: take prediction using each model and then average it
        """
        
        prediction_result = []
        
        for model in self.model:
            prediction_result.append(self.make_prediction(row, nodeno, model))
        
        #group average: this level=0 groupby function with concat takes every result dataframes 
        #in array and mean rows having same index in each dataframe
        #So finally we can get the averaged probability for each row, each column
        return pd.concat(prediction_result).groupby(level=0).mean()
        
    def predict(self, df):
        
        #Hint 1: drop id and class
        df_copy = df.copy()
        
        #it is better to check it seperately because in some unknown dataset there can be only one among ID and CLASS
        if "ID" in df_copy.columns:
            df_copy.drop(["ID"], inplace=True, axis=1)
        if "CLASS" in df_copy.columns:
            df_copy.drop(["CLASS"], inplace=True, axis=1)
            
        #Hint 1: apply imputation, binning
        df_copy = apply_imputation(df_copy, self.imputation)
        df_copy = apply_bins(df_copy, self.binning)
        
        #Hint 2: iterate over the rows
        predictions = df_copy.apply(self.make_prediction_average, axis=1, nodeno=0)
        
        #fix NaN value (zero probabilities) into real zero
        predictions.fillna(0, inplace=True)
        
        return predictions

### Our result

In [27]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 10.50 s.
Testing time (1, 1): 0.20 s.
Training time (1, 2): 11.95 s.
Testing time (1, 2): 0.20 s.
Training time (1, 5): 15.39 s.
Testing time (1, 5): 0.19 s.
Training time (2, 1): 10.03 s.
Testing time (2, 1): 0.19 s.
Training time (2, 2): 11.97 s.
Testing time (2, 2): 0.19 s.
Training time (2, 5): 15.42 s.
Testing time (2, 5): 0.20 s.
Training time (5, 1): 6.75 s.
Testing time (5, 1): 0.20 s.
Training time (5, 2): 7.80 s.
Testing time (5, 2): 0.20 s.
Training time (5, 5): 11.03 s.
Testing time (5, 5): 0.20 s.


Accuracy  Brier score       AUC
1 1  0.691589     0.429096  0.884213
  2  0.644860     0.444150  0.864234
  5  0.616822     0.451942  0.853692
2 1  0.682243     0.420095  0.893244
  2  0.654206     0.442932  0.885548
  5  0.719626     0.388174  0.913320
5 1  0.654206     0.422594  0.874119
  2  0.663551     0.451512  0.870941
  5  0.700935     0.435951  0.880581

### Henrik's result

In [18]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 7.51 s.
Testing time (1, 1): 0.09 s.
Training time (1, 2): 10.65 s.
Testing time (1, 2): 0.09 s.
Training time (1, 5): 19.78 s.
Testing time (1, 5): 0.09 s.
Training time (2, 1): 7.97 s.
Testing time (2, 1): 0.09 s.
Training time (2, 2): 12.01 s.
Testing time (2, 2): 0.10 s.
Training time (2, 5): 19.42 s.
Testing time (2, 5): 0.10 s.
Training time (5, 1): 4.57 s.
Testing time (5, 1): 0.10 s.
Training time (5, 2): 7.31 s.
Testing time (5, 2): 0.09 s.
Training time (5, 5): 9.75 s.
Testing time (5, 5): 0.09 s.


Accuracy  Brier score       AUC
1 1  0.644860     0.477912  0.860736
  2  0.710280     0.411981  0.892842
  5  0.710280     0.421511  0.895716
2 1  0.663551     0.452421  0.872447
  2  0.626168     0.411684  0.911137
  5  0.654206     0.435847  0.881621
5 1  0.644860     0.457859  0.865636
  2  0.719626     0.410616  0.903016
  5  0.616822     0.455782  0.883897

In [24]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.11


### Comment on assumptions, things that do not work properly, etc.

We made every algorithm correctly.
As it is random, and test case is really small, accuracy and other metrics are different in every trial.

We marked new logics in random forest with ##### line